In [1]:
import pandas as pd
import numpy as np

In [2]:
def load_data_in_chunks(file_path, chunk_size=50000):
    chunks = []
    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        chunks.append(chunk)
    return pd.concat(chunks, axis=0)

# Load data
df = load_data_in_chunks('data/raw/merged.csv')

# Display the first few rows for inspection
df.head()

,legId,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isBasicEconomy,isRefundable,isNonStop,totalFare,...,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,e1b137527b9175d7d930c3af82e70ae0,2022-04-19,2022-05-20,OAK,ATL,PT7H52M,False,False,False,103.98,...,1653107460||1653126600,2022-05-20T22:31:00.000-06:00||2022-05-21T05:5...,DEN||ATL,OAK||DEN,Frontier Airlines||Frontier Airlines,F9||F9,||Airbus A320,9180||10620,943||1207,coach||coach
1,d813ebd107e3fa700206c0d96015da7a,2022-04-19,2022-05-20,OAK,ATL,PT6H15M,False,False,False,216.58,...,1653067080||1653084660,2022-05-20T10:18:00.000-07:00||2022-05-20T18:1...,LAX||ATL,OAK||LAX,Spirit Airlines||Spirit Airlines,NK||NK,||AIRBUS INDUSTRIE A320 SHARKLETS,4920||15600,None||None,coach||coach
2,e8ece5ad6f5962c696e06e031fc2a24a,2022-04-19,2022-05-20,OAK,ATL,PT9H6M,False,False,False,216.58,...,1653056820||1653084660,2022-05-20T07:27:00.000-07:00||2022-05-20T18:1...,LAX||ATL,OAK||LAX,Spirit Airlines||Spirit Airlines,NK||NK,AIRBUS INDUSTRIE A320 SHARKLETS||AIRBUS INDUST...,4920||15600,None||None,coach||coach
3,c004a54681335100f326c9613b3c9448,2022-04-19,2022-05-20,OAK,ATL,PT6H17M,False,False,False,237.58,...,1653110940||1653127980,2022-05-20T22:29:00.000-07:00||2022-05-21T06:1...,LAS||ATL,OAK||LAS,Spirit Airlines||Spirit Airlines,NK||NK,AIRBUS INDUSTRIE A320 SHARKLETS||Airbus A319,5580||13980,None||None,coach||coach
4,4a42bbf77211b4afa7b9e14005949120,2022-04-19,2022-05-20,OAK,ATL,PT14H12M,False,False,False,307.21,...,1653115560||1653159180,2022-05-20T23:46:00.000-07:00||2022-05-21T14:5...,SEA||ATL,OAK||SEA,Alaska Airlines||Alaska Airlines,AS||AS,Boeing 737-900||Boeing 737-900,7500||17580,672||2178,coach||coach


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13519999 entries, 0 to 13519998
Data columns (total 23 columns):
 #   Column                             Dtype  
---  ------                             -----  
 0   legId                              object 
 1   searchDate                         object 
 2   flightDate                         object 
 3   startingAirport                    object 
 4   destinationAirport                 object 
 5   travelDuration                     object 
 6   isBasicEconomy                     bool   
 7   isRefundable                       bool   
 8   isNonStop                          bool   
 9   totalFare                          float64
 10  totalTravelDistance                float64
 11  segmentsDepartureTimeEpochSeconds  object 
 12  segmentsDepartureTimeRaw           object 
 13  segmentsArrivalTimeEpochSeconds    object 
 14  segmentsArrivalTimeRaw             object 
 15  segmentsArrivalAirportCode         object 
 16  segmentsDepartur

In [4]:
# Check for missing values
missing_values = df.isnull().sum()
missing_values

legId                                     0
searchDate                                0
flightDate                                0
startingAirport                           0
destinationAirport                        0
travelDuration                            0
isBasicEconomy                            0
isRefundable                              0
isNonStop                                 0
totalFare                                 0
totalTravelDistance                  959619
segmentsDepartureTimeEpochSeconds         0
segmentsDepartureTimeRaw                  0
segmentsArrivalTimeEpochSeconds           0
segmentsArrivalTimeRaw                    0
segmentsArrivalAirportCode                0
segmentsDepartureAirportCode              0
segmentsAirlineName                       0
segmentsAirlineCode                       0
segmentsEquipmentDescription         262676
segmentsDurationInSeconds                 0
segmentsDistance                          0
segmentsCabinCode               

In [5]:
df.dropna(inplace=True)

In [6]:
# Define preprocessing function
def preprocessor(df):

    def extract_segment(s, position):
        segments = s.strip().split('||')
        if position == "first":
            return segments[0]
        elif position == "last":
            return segments[-1]
        else:
            return s

    # Helper function to parse the timestamp
    def parse_timestamp(ts):
        date, time_zone = ts.split('T')
        time, time_zone = time_zone.split('.')
        time = time.split('+')[0] if '+' in time else time.split('-')[0] if '-' in time else time

        # Extract hour from the time
        hour = int(time.split(':')[0])

        return date, time, time_zone, hour


    def is_float(value):
        """Check if the string can be converted to a float."""
        try:
            float(value)
            return True
        except ValueError:
            return False

    def convert_to_floats(segment_list):
        """Convert a list of strings to floats, ignoring invalid strings."""
        return [float(val) for val in segment_list if val and is_float(val)]

    def compute_sum_for_segment(segment_str, separator='|'):
        """Compute the sum for a given segment string."""

        # Check if the segment_str is not a string
        if not isinstance(segment_str, str):
            print(f"Unexpected type: {type(segment_str)}")
            return segment_str  # Return the original value

        segments = segment_str.split(separator)
        return sum(convert_to_floats(segments))

    def compute_summation_optimized(df, sum_cols):
        """Optimized function to compute the summation for specific columns."""
        for col in sum_cols:
            df[col] = df[col].apply(compute_sum_for_segment)
        return df


    # Extracting the first segment's data for departures
    df['segmentsDepartureTimeRaw'] = df['segmentsDepartureTimeRaw'].apply(lambda x: extract_segment(x, "first"))
    df['segmentsDepartureAirportCode'] = df['segmentsDepartureAirportCode'].apply(lambda x: extract_segment(x, "first"))

    # Extracting the last segment's data for arrivals
    df['segmentsArrivalTimeRaw'] = df['segmentsArrivalTimeRaw'].apply(lambda x: extract_segment(x, "last"))
    df['segmentsArrivalAirportCode'] = df['segmentsArrivalAirportCode'].apply(lambda x: extract_segment(x, "last"))

    # Apply the parsing for 'segmentsDepartureTimeRaw'
    df['departure_date'], df['departure_time'], df['departure_time_zone'], df['departure_hour'] = zip(*df['segmentsDepartureTimeRaw'].apply(parse_timestamp))

    # Apply the parsing for 'segmentsArrivalTimeRaw'
    df['arrival_date'], df['arrival_time'], df['arrival_time_zone'], df['arrival_hour'] = zip(*df['segmentsArrivalTimeRaw'].apply(parse_timestamp))

     # Drop specified columns
    df = df.drop(columns=['segmentsDepartureTimeRaw','segmentsArrivalTimeRaw',
                          'segmentsDepartureTimeEpochSeconds','segmentsArrivalTimeEpochSeconds',
                          'segmentsAirlineName','arrival_date','arrival_time_zone','arrival_time',
                          'departure_date','departure_time_zone','departure_time'
                         ], errors='ignore')

    # Apply the optimized summation computation
    df = compute_summation_optimized(df,['segmentsDurationInSeconds', 'segmentsDistance'])

    return df

In [7]:
# Apply preprocessing function to the DataFrame
df_cleaned = preprocessor(df.copy())

# Display the first few rows of the preprocessed DataFrame
df_cleaned.head()

,legId,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isBasicEconomy,isRefundable,isNonStop,totalFare,totalTravelDistance,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode,departure_hour,arrival_hour
0,e1b137527b9175d7d930c3af82e70ae0,2022-04-19,2022-05-20,OAK,ATL,PT7H52M,False,False,False,103.98,2150.0,ATL,OAK,F9||F9,||Airbus A320,19800.0,2150.0,coach||coach,18,5
4,4a42bbf77211b4afa7b9e14005949120,2022-04-19,2022-05-20,OAK,ATL,PT14H12M,False,False,False,307.21,2850.0,ATL,OAK,AS||AS,Boeing 737-900||Boeing 737-900,25080.0,2850.0,coach||coach,21,14
5,9157572cdd530976f707a162a4fcfde8,2022-04-19,2022-05-20,OAK,ATL,PT9H8M,True,False,False,318.60,2850.0,ATL,OAK,AS||AS,Boeing 737-800||Boeing 737-900,25440.0,2850.0,coach||coach,6,18
7,e8baed3d41a0c35856f7591a5baad060,2022-04-19,2022-05-20,OAK,ATL,PT13H14M,False,False,False,410.69,2285.0,ATL,OAK,DL||DL||DL,Embraer 175 (Enhanced Winglets)||Boeing 737-80...,22560.0,2285.0,coach||coach||coach,16,8
8,965e29ef7e8d11bf05f723a28ab8e4e0,2022-04-19,2022-05-20,OAK,ATL,PT12H40M,False,False,False,437.71,2477.0,ATL,OAK,DL||DL||DL,Embraer 175 (Enhanced Winglets)||Airbus A220-1...,24960.0,2477.0,coach||coach||coach,17,8


In [8]:
df_cleaned.to_csv('data/processed/cleaned.csv')

In [31]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12337897 entries, e1b137527b9175d7d930c3af82e70ae0 to 246de99f674d932434ec729095931800
Data columns (total 23 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   startingAirport               int32  
 1   destinationAirport            int32  
 2   travelDuration                object 
 3   isBasicEconomy                int64  
 4   isRefundable                  int64  
 5   isNonStop                     int64  
 6   totalFare                     float64
 7   totalTravelDistance           float64
 8   segmentsArrivalAirportCode    int32  
 9   segmentsDepartureAirportCode  int32  
 10  segmentsAirlineCode           object 
 11  segmentsEquipmentDescription  object 
 12  segmentsDurationInSeconds     float64
 13  segmentsDistance              float64
 14  segmentsCabinCode             object 
 15  departure_hour                int64  
 16  arrival_hour                  int64  
 17  search Day   

In [10]:
print(df_cleaned.describe())

          totalFare  totalTravelDistance  segmentsDurationInSeconds  \
count  1.233790e+07         1.233790e+07               1.233790e+07   
mean   3.836261e+02         1.572339e+03               1.663993e+04   
std    2.102343e+02         8.412266e+02               6.821145e+03   
min    2.397000e+01         9.700000e+01               2.760000e+03   
25%    2.301000e+02         8.620000e+02               1.140000e+04   
50%    3.566000e+02         1.392000e+03               1.596000e+04   
75%    4.986000e+02         2.375000e+03               2.190000e+04   
max    8.260610e+03         4.430000e+03               4.596000e+04   

       segmentsDistance  departure_hour  arrival_hour  
count      1.233790e+07    1.233790e+07  1.233790e+07  
mean       1.574434e+03    1.204657e+01  1.538328e+01  
std        8.416893e+02    5.168067e+00  5.665138e+00  
min        9.700000e+01    0.000000e+00  0.000000e+00  
25%        8.620000e+02    8.000000e+00  1.200000e+01  
50%        1.393000e+03 

In [2]:
# read new csv to save time
def load_data_in_chunks(file_path, chunk_size=50000):
    chunks = []
    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        chunks.append(chunk)
    return pd.concat(chunks, axis=0)

# Load data
df_cleaned = load_data_in_chunks('data/processed/cleaned.csv')

In [3]:
df_cleaned = df_cleaned.set_index(['legId'])

In [4]:
# DATA PREP

In [5]:
# Change date values
df_cleaned['searchDate'] = pd.to_datetime(df_cleaned['searchDate'])
df_cleaned['search Day'] = df_cleaned['searchDate'].dt.day
df_cleaned['search Month'] = df_cleaned['searchDate'].dt.month
df_cleaned['search Year'] = df_cleaned['searchDate'].dt.year

In [6]:
# Change date values
df_cleaned['flightDate'] = pd.to_datetime(df_cleaned['flightDate'])
df_cleaned['flight Day'] = df_cleaned['flightDate'].dt.day
df_cleaned['flight Month'] = df_cleaned['flightDate'].dt.month
df_cleaned['flight Year'] = df_cleaned['flightDate'].dt.year

In [7]:
# Function to keep only the first value before '||'
def keep_first_value(value):
    split_result = value.split('||')
    return pd.Series({'cabinType': split_result[0]})

# Apply the custom function to keep the first value and create a new column
new_column = df_cleaned['segmentsCabinCode'].apply(keep_first_value)

# Concatenate the new column with the original DataFrame and drop the original column
df_cleaned = pd.concat([df_cleaned, new_column], axis=1)


KeyboardInterrupt



In [ ]:
df_cleaned

In [7]:
from sklearn.preprocessing import LabelEncoder
# columns to encode
columns_to_encode = ['startingAirport', 'destinationAirport', 'isBasicEconomy',	'isRefundable', 'isNonStop', 'segmentsArrivalAirportCode', 'segmentsDepartureAirportCode', 'cabinType']

# Create a LabelEncoder instance
label_encoder = LabelEncoder()

# Loop through the specified columns and apply label encoding
for column in columns_to_encode:
    df_cleaned[column] = label_encoder.fit_transform(df_cleaned[column])


In [8]:
# Drop columns
columns_to_drop = ['searchDate', 'flightDate', 'travelDuration', 'Unnamed: 0', 'segmentsAirlineCode', 'segmentsEquipmentDescription', 'segmentsCabinCode']
df_cleaned = df_cleaned.drop(columns=columns_to_drop)

In [ ]:
df_cleaned

In [10]:
X = df_cleaned.drop('totalFare', axis=1)
y = df_cleaned['totalFare']

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import accuracy_score
# Split the dataset into a training set and a testing set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [12]:
# Create a KNN regression model with a specified number of neighbors (e.g., 3)
k = 3
knn = KNeighborsRegressor(n_neighbors=k)

In [13]:
# Fit the KNN model to the training data
knn.fit(X_train, y_train)

ValueError: could not convert string to float: 'coach||coach'

In [ ]:
# Make predictions on the test data
y_pred = knn.predict(X_val)

In [ ]:
# Evaluate the model using a regression metric, such as Mean Squared Error (MSE)
mse = mean_squared_error(y_val, y_pred)
print(f'Mean Squared Error: {mse:.2f}')